In [1]:

import numpy as np
from time import sleep
import googlemaps
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
load_dotenv
maps_token = getpass.getpass("maps_token: ")
map_client = googlemaps.Client(maps_token)

In [115]:
def extracting_data(business,keyword, radius):
    sleep(1)
    location = (37.4681237,-122.1547526)  
    radius = radius # radius in meters
    business = business
    keyword = keyword
    results = []
    response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword)
    results.extend(response['results'])
    while True:
        try:
            sleep(2)
            response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword, page_token=next_page)
            results.extend(response['results'])
            next_page = response['next_page_token']
        except:
            break
    return results    

In [116]:
def dataframe(data, business, keyword, color, icon_color, icon):
    list_local = []
    df = pd.DataFrame(columns = ['Category','Type', 'Name', 'Address', 'Latitude', 'Longitude', 'Icon', 'Icon Color', 'Color', 'Prefix'])
    for result in data:
        local = {}
        local['Category'] = business
        local['Type'] = keyword
        local['Color'] = color
        local['Icon Color'] = icon_color
        local['Icon'] = icon
        local['Prefix'] = 'fa'
        local['Name'] = result['name']
        local['Address'] = result['vicinity']
        local['Latitude'] = result['geometry']['location']['lat']
        local['Longitude'] = result['geometry']['location']['lng']
        list_local.append(local)
    df = pd.DataFrame(list_local)
    return df    

In [117]:
bars = extracting_data('Bar', 'Bar', 1500)

In [118]:
restaurant = extracting_data('Restaurant', 'Restaurant', 1000)

In [119]:
school = extracting_data('School', 'public School', 2000)

In [120]:
veg_restaurant = extracting_data('Restaurant', 'vegetarian', 500)

In [121]:
hairdresser = extracting_data('hairdresser', 'hairdresser', 1000)

In [122]:
tech_startups = extracting_data('tech startups', 'startups', 2000)

In [123]:
coffee = extracting_data('Coffe', 'Coffee Shop', 1500)

In [124]:
metro = extracting_data('Metro', 'Metro Stations', 700)

In [125]:
bus = extracting_data('Bus', 'Bus Station', 700)

In [126]:
travel_agency = extracting_data('Travel Agency', 'Travel Agency', 1500)

In [127]:
df_bars_us = dataframe(bars,'Bar', 'Bar', 'red', 'white', 'beer-mug-empty' )

In [128]:
df_restaurants_us = dataframe(restaurant,'Restaurant', 'Restaurant', 'orange', 'white', 'utensils')

In [129]:
df_school_us = dataframe(school, 'School', 'Public School', 'blue', 'black', 'graduation-cap')

In [130]:
df_veg_us = dataframe(veg_restaurant,'Restaurant', 'Vegetarian', 'green', 'white', 'wheat-awn')

In [131]:
df_hairdresser_us = dataframe(hairdresser, 'Hairdresser', 'Health & Beauty', 'white', 'black', 'scissors')

In [132]:
df_tech_startups_us = dataframe(tech_startups, 'Tech startups', 'Startups', 'black', 'white', 'desktop')

In [133]:
df_coffee_us = dataframe(coffee, 'Coffee', 'Coffee  Shop', 'beige', 'back', 'mug-hot')

In [134]:
df_metro_us = dataframe(metro, 'Metro', 'Metro Station', 'darkred', 'white', 'train-subway')

In [135]:
df_bus_us = dataframe(bus, 'Bus', 'Bus Station', 'darkblue', 'white', 'bus')

In [136]:
df_travel_agency_us = dataframe(travel_agency, 'Travel Agency', 'Travel Agency', 'purple', 'black', 'plane')

# Exporting the DataFrame to Mongodb

In [137]:
def export_to_mongodb(data):
    client = MongoClient("localhost", 27017)
    db = client["Project-Spot"]
    for i, v in enumerate(data):
        file_csv = f"Project-Spot_{i}.csv"
        v.to_csv(f'data/{file_csv}', index=False)
        collection_name = v['Type'].iloc[0]
        collection = db[collection_name]
        result = v.to_dict(orient='records')
        collection.insert_many(result)
    

# Create the Map layer and add it to the FeatureGroup

In [138]:
city_map = [
    df_coffee_us, df_tech_startups_us,
    df_hairdresser_us, df_veg_us, 
    df_school_us, df_bars_us,
    df_restaurants_us, df_metro_us,
    df_bus_us, df_travel_agency_us
]

layers_name = [
    'Coffee Shop', 'Startups',
    'Health & Beauty', 'Vegetarian',
    'Public School', 'Bar',
    'Restaurant', 'Metro Station',
    'Bus Station', 'Travel Agency'
]

In [139]:
export_to_mongodb(city_map)

KeyError: 'Type'

# Creating the MAP

In [144]:
def add_spot(data, layer):
    main_map = folium.Map(location = ['37.4681237','-122.1547526'], tiles="Cartodb dark_matter",zoom_start=15)
    layer_list = []
    red_cicle = folium.FeatureGroup(name = 'Red Cicle')    
    folium.Circle(location=['37.4681237','-122.1547526'], radius=2500, color='red', fill=True).add_to(red_cicle)
    layer_list.append(red_cicle)
    yellow_cicle = folium.FeatureGroup(name = 'Yellow Cicle')    
    folium.Circle(location=['37.4681237','-122.1547526'], radius=1500, color='yellow', fill=True).add_to(yellow_cicle)
    layer_list.append(yellow_cicle)
    green_cicle = folium.FeatureGroup(name = 'Green Cicle')    
    folium.Circle(location=['37.4681237','-122.1547526'], radius=700, color='green', fill=True).add_to(green_cicle)
    layer_list.append(green_cicle)
    for df, lay in zip(data, layer):
        layer = folium.FeatureGroup(name = f'{lay}')
        for k, v in df.iterrows():
            icon = folium.Icon(
                color= v['Color'], 
                icon_color= v['Icon Color'], 
                icon= v['Icon'], prefix= v['Prefix']
                )
            folium.Marker(location=[v['Latitude'], v['Longitude']], popup= v['Name'], icon = icon).add_to(layer)
        layer_list.append(layer)
    for v in layer_list:
        v.add_to(main_map)
    folium.LayerControl().add_to(main_map)
    main_map.save('usa_map.html')
    return main_map

In [145]:
add_spot(city_map, layers_name)